In [ ]:
import ffmpy
import pydub
from pydub import AudioSegment, playback
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import random


import utils
from utils import mkdir, find_wavs, read_audio_from_dir

Input files:

- `media/male/`: Words read by a male speaker, manually edited to all be approximately the same length. 
- `media/female/`: Targets read by a male speaker, manually edited to all be approximately the same length. 

Recordings in `male` are shorter than in `female`.


In [ ]:
sample_dir = 'media/'
target_dir = os.path.join(sample_dir, 'female')
source_dir = os.path.join(sample_dir, 'male')
compressed_dir = os.path.join(sample_dir, 'compressed')
reversed_dir = os.path.join(sample_dir, 'reversed')
for d in [compressed_dir, reversed_dir]:
    mkdir(d)

# Step 1 - Compress Male Speaker and Make Reversed versions

In [ ]:
compression_rate = 1.
from utils import compress, reverse, rm
file_names = find_wavs(source_dir)
file_names

In [ ]:
for fn in file_names:
    inp = os.path.join(source_dir, fn)
    out = os.path.join(compressed_dir, fn)
    out_r = os.path.join(reversed_dir, fn)
    rm(out)
    rm(out_r)
    compress(inp, out, amount=compression_rate)
    reverse(out, out_r)

# Step 2 - Join together primes and target audio

In [ ]:
silence = pydub.AudioSegment.silent(duration=200)
targets = read_audio_from_dir(target_dir)
compressed = read_audio_from_dir(compressed_dir)
reversals = read_audio_from_dir(reversed_dir)
from utils import play

In [ ]:
for key, audio in reversals.items():
    print(key)
    play(audio)
    play(silence)

In [ ]:
for key, audio in targets.items():
    print(key)
    play(audio)

In [ ]:
masks = {k : v for k, v in reversals.items() if k not in ['Left', 'Right']}
mask_list = [v for k, v in masks.items()]
mask_list

In [ ]:
primes = {k : v for k, v in compressed.items() if k in ['Left', 'Right']}
primes

In [ ]:
for key, audio in primes.items():
    print(key)
    play(audio)
    play(silence)

In [ ]:
def concatenate_words(words: list, silence : pydub.AudioSegment = silence, ISI=200) -> pydub.AudioSegment:
    '''Join together a list of recordings, with silence in between
    ISI is time between onsets of consecutive primes or masks.
    '''
    lengths = [len(w) for w in words]
    result = silence[0:ISI]
    for w in words:
        if len(w) > ISI:
            w = w[:ISI]
        w = w.fade_in(10)
        w = w.fade_out(10)
        dur = len(w)
        result = result + w + silence[:(ISI-dur)]
    return result


In [ ]:
def generate_sequence(prime_loc: int, prime_label: str, target_label: str, 
                      boost = 1,
                      mask=True, ISI=150, pad_end=3,
                      targets: dict = targets,
                      primes: dict=primes,
                      masks: list=mask_list) -> pydub.AudioSegment:
    '''Generate audio for a single trial
    Args:
        prime_loc: Number of masks before (and including) the prime
        prime_label: 'Left' or 'Right'
        target_label: 'Left' or 'Right'
        mask: Should the prime be masked?
        ISI: Time between primes/masks
        pad_end: Number of masks after the prime
        targets: dict of AudioSegments, with keys 'Left' and 'Right'
        primes: As above
        masks: List of mask AudioSegments. We don't care about order, so no keys.
        
    Returns:
        One long pydub.AudioSegment
    '''
    n = prime_loc + pad_end
    if mask:
        ix = np.random.choice(range(len(masks)), n)
        mask_audio = [masks[i] for i in ix]        
    else:
        mask_audio = [silence] * (n + 5)
    if prime_label is not None:
        mask_audio[prime_loc] = primes[prime_label]
    mask_audio = mask_audio + [silence] * 3
    if boost > 0:
        result = concatenate_words(mask_audio, silence, ISI=ISI) - boost
    if target_label is not None:
        target_audio = targets[target_label]
        if boost < 0:
            target_audio += boost
        target_onset = (prime_loc+2) * (ISI) # First 1xISI is silence.
        result = result.overlay(target_audio, position=target_onset)
    return result

In [ ]:
primes

## Manually specify some trials

In [ ]:
seq = generate_sequence(5, 'Right', 'Right', mask=False)
play(seq)

In [ ]:
seq = generate_sequence(5, 'Right', None, mask=True)
play(seq)

In [ ]:
# seq = generate_sequence(5, 'Right', 'Right', mask=True)
# play(seq)

In [ ]:
# seq = generate_sequence(5, 'Right', 'Right', mask=True)
# play(seq)

In [ ]:
# seq = generate_sequence(5, 'Right', None, mask=True)
# play(seq)

## Automatically generate and save some trials

In [ ]:
def do_trial(prime=None, mask=None, target=None, save=None):
    '''Generate a trial AudioSegment, and either save or play it.
    If prime, and target should be either None, 'Left', or 'Right'
    Mask should be either True or False (None will be coerced to False)
    '''
    LR = ['Left', 'Right']
    LRN = ['Left', 'Right', None]
    n = np.random.randint(4, 10)
    assert prime in LRN
    assert target in LRN
    seq = generate_sequence(n, prime, target, mask=mask)
    if save is not None:
        seq.export(out_f=save, format='wav')
    else:
        play(seq)

In [ ]:
%mkdir -p output

In [ ]:
do_trial(mask=False, prime=None, target='Left', save='output/target_left.wav') # Target only

In [ ]:
do_trial(mask=False, prime=None, target='Right', save='output/target_right.wav') # Target only

In [ ]:
LR = ['Left', 'Right']
for prime in LR:
    for target in LR:
        fn = ('output/prime_%s_target_%s.wav' % (prime, target)).lower()
        do_trial(mask=False, prime=prime, target=target, save=fn)

In [ ]:
LR = ['Left', 'Right']
for prime in LR:
    for target in LR:
        fn = ('output/prime_%s_target_%s_masked.wav' % (prime, target)).lower()
        do_trial(mask=True, prime=prime, target=target, save=fn)

In [ ]:
do_trial(mask=False, prime='Left', target='Right', save='output/prime_left_target_left.wav')
do_trial(mask=False, prime=None, target='Right', save='output/prime_right_target_right.wav')

In [ ]:
do_trial(mask=True, prime='Left', target='Right') # Target and prime

In [ ]:
do_trial(mask=True, prime=None, target=None) # Target and prime

In [ ]:
do_trial(mask=True, prime='Right', target=None) # Target and prime

In [ ]:
# do_trial(mask=False, prime=True, target=True) # Target and prime
do_trial(mask=False, prime='Right', target='Left') # Target and prime
# do_trial(mask=False, prime=True, target=True) # Target and prime